# 12장. 군집분석 : 타깃마케팅을 위한 K-평균 군집화

### 1) 데이터 수집

In [1]:
import pandas as pd
import math

ModuleNotFoundError: No module named 'pandas'

In [ ]:
retail_df = pd.read_excel('data/Online_Retail.xlsx')
retail_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,40513.351389,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,40513.351389,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,40513.351389,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,40513.351389,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,40513.351389,3.39,17850.0,United Kingdom


### 2) 데이터 준비 및 탐색

In [ ]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  float64
 5   Price        541909 non-null  float64
 6   Customer ID  406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
print(retail_df.isnull().sum())

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64


In [ ]:
# 오류 데이터 정제
retail_df = retail_df[retail_df['Quantity'] > 0] 
#retail_df = retail_df[retail_df['Price'] > 0] 
retail_df = retail_df[retail_df['Customer ID'].notnull()] 

# 'Customer ID' 자료형을 정수형으로 변환
retail_df['Customer ID'] = retail_df['Customer ID'].astype(int)

In [ ]:
retail_df.info()
print(retail_df.isnull().sum())
print(retail_df.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      397924 non-null  object 
 1   StockCode    397924 non-null  object 
 2   Description  397924 non-null  object 
 3   Quantity     397924 non-null  int64  
 4   InvoiceDate  397924 non-null  float64
 5   Price        397924 non-null  float64
 6   Customer ID  397924 non-null  int32  
 7   Country      397924 non-null  object 
dtypes: float64(2), int32(1), int64(1), object(4)
memory usage: 25.8+ MB
Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64
(397924, 8)


In [ ]:
# 중복 레코드 제거
retail_df.drop_duplicates(inplace=True)
print(retail_df.shape) #작업 확인용 출력

(392732, 8)


#### - 제품 수, 거래건 수, 고객 수 탐색

In [ ]:
pd.DataFrame([{'Product':len(retail_df['StockCode'].value_counts()),
              'Transaction':len(retail_df['Invoice'].value_counts()),
              'Customer':len(retail_df['Customer ID'].value_counts())}], 
             columns = ['Product', 'Transaction', 'Customer'],
            index = ['counts'])

,Product,Transaction,Customer
counts,3665,18536,4339


In [ ]:
retail_df['Country'].value_counts()

United Kingdom          349227
Germany                   9027
France                    8327
EIRE                      7228
Spain                     2480
Netherlands               2363
Belgium                   2031
Switzerland               1842
Portugal                  1453
Australia                 1184
Norway                    1072
Italy                      758
Channel Islands            747
Finland                    685
Cyprus                     603
Sweden                     450
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     245
Unspecified                241
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon                     45
Lithuani

In [ ]:
# 주문금액 컬럼 추가
retail_df['SaleAmount'] = retail_df['Price'] * retail_df['Quantity']
retail_df.head() #작업 확인용 출력

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,SaleAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,40513.351389,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,40513.351389,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,40513.351389,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,40513.351389,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,40513.351389,3.39,17850,United Kingdom,20.34


#### - 고객의 마지막 주문후 경과일(Elapsed Days), 주문횟수(Freq), 주문 총액(Total Amount) 구하기

In [ ]:
aggregations = {    
    'Invoice':'count', #주문건수 conunt
    'SaleAmount':'sum', #문금액 sum
    'InvoiceDate':'max' #주문일 max
}

customer_df = retail_df.groupby('Customer ID').agg(aggregations)
customer_df = customer_df.reset_index()

customer_df.head()  #작업 확인용 출력

,Customer ID,Invoice,SaleAmount,InvoiceDate
0,12346,1,77183.60,40561.417361
1,12347,182,4310.00,40884.661111
2,12348,31,1797.24,40811.550694
3,12349,73,1757.55,40868.410417
4,12350,17,334.40,40576.667361


In [ ]:
# 컬럼이름 바꾸기
customer_df = customer_df.rename(columns = {'Invoice':'Freq', 'InvoiceDate':'ElapsedDays'})
customer_df.head() #작업 확인용 출력

,Customer ID,Freq,SaleAmount,ElapsedDays
0,12346,1,77183.60,40561.417361
1,12347,182,4310.00,40884.661111
2,12348,31,1797.24,40811.550694
3,12349,73,1757.55,40868.410417
4,12350,17,334.40,40576.667361


#### - 마지막 구매후 경과일 계산하기

In [ ]:
import datetime

customer_df['ElapsedDays'] = datetime.datetime(2011,12,20) - customer_df['ElapsedDays']
customer_df.head() #작업 확인용 출력

TypeError: unsupported operand type(s) for -: 'Timestamp' and 'float'

In [ ]:
customer_df['ElapsedDays'] = customer_df['ElapsedDays'].apply(lambda x: x.days+1)
customer_df.head() #작업 확인용 출력

#### - 현재 데이터 값의 분포 확인하기

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots()
ax.boxplot([customer_df['Freq'], customer_df['SaleAmount'], customer_df['ElapsedDays']], sym='bx')
plt.xticks([1, 2, 3], ['Freq', 'SaleAmount','ElapsedDays' ])
plt.show()

#### - 데이터 값의 왜곡(치우침)을 줄이기 위한 작업 : 로그 함수로 분포 조정

In [ ]:
import numpy as np

customer_df['Freq_log'] = np.log1p(customer_df['Freq'])
customer_df['SaleAmount_log'] = np.log1p(customer_df['SaleAmount'])
customer_df['ElapsedDays_log'] = np.log1p(customer_df['ElapsedDays'])

customer_df.head()  #작업 확인용 출력

In [ ]:
# 조정된 데이터 분포를 다시 박스플롯으로 확인하기
fig, ax = plt.subplots()
ax.boxplot([customer_df['Freq_log'], customer_df['SaleAmount_log'],customer_df['ElapsedDays_log']], 
           sym='bo')
plt.xticks([1, 2, 3], ['Freq_log', 'SaleAmount_log', 'ElapsedDays_log'])
plt.show()

### 3) 모델 구축 : K-평균 군집화 모델 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

In [ ]:
X_features = customer_df[['Freq_log', 'SaleAmount_log', 'ElapsedDays_log']].values
print(X_features)

In [ ]:
# 정규 분포로 다시 스케일링하기
from sklearn.preprocessing import StandardScaler

X_features_scaled = StandardScaler().fit_transform(X_features)

### - 최적의 k 찾기 (1) 엘보우 방법

In [ ]:
distortions = []

for i in range(1, 11):
    kmeans_i = KMeans(n_clusters=i, random_state=0)  # 모델 생성
    kmeans_i.fit(X_features_scaled)   # 모델 훈련
    distortions.append(kmeans_i.inertia_)
    
plt.plot(range(1,11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0) # 모델 생성

# 모델 학습과 결과 예측(클러스터 레이블 생성)
Y_labels = kmeans.fit_predict(X_features_scaled) 

In [ ]:
customer_df['ClusterLabel'] = Y_labels

customer_df  #작업 확인용 출력

## 4) 결과 분석 및 시각화

### - 최적의 k 찾기 (2) 실루엣 계수에 따른 각 클러스터의 비중 시각화 함수 정의

In [ ]:
from matplotlib import cm

def silhouetteViz(n_cluster, X_features): 
    kmeans = KMeans(n_clusters=n_cluster, random_state=0)
    Y_labels = kmeans.fit_predict(X_features)
    silhouette_values = silhouette_samples(X_features, Y_labels, metric='euclidean')
    y_ax_lower, y_ax_upper = 0, 0
    y_ticks = []

    for c in range(n_cluster):
        c_silhouettes = silhouette_values[Y_labels == c]
        c_silhouettes.sort()
        y_ax_upper += len(c_silhouettes)
        color = cm.jet(float(c) / n_cluster)
        plt.barh(range(y_ax_lower, y_ax_upper), c_silhouettes, height=1.0, edgecolor='none', color=color)
        y_ticks.append((y_ax_lower + y_ax_upper) / 2.)
        y_ax_lower += len(c_silhouettes)
    
    silhouette_avg = np.mean(silhouette_values)
    plt.axvline(silhouette_avg, color='red', linestyle='--')
    plt.title('Number of Cluster: '+ str(n_cluster)+'\n' \
              + 'Silhouette Score: '+ str(round(silhouette_avg,3)))
    plt.yticks(y_ticks, range(n_cluster))   
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette coefficient')
    plt.tight_layout()
    plt.show()

### - 클러스터 수에 따른 클러스터 데이터 분포의 시각화 함수 정의

In [ ]:
def clusterScatter(n_cluster, X_features): 
    c_colors = []
    kmeans = KMeans(n_clusters=n_cluster, random_state=0)
    Y_labels = kmeans.fit_predict(X_features)

    for i in range(n_cluster):
        c_color = cm.jet(float(i) / n_cluster) #클러스터의 색상 설정
        c_colors.append(c_color)
        #클러스터의 데이터 분포를 동그라미로 시각화
        plt.scatter(X_features[Y_labels == i,0], X_features[Y_labels == i,1],
                     marker='o', color=c_color, edgecolor='black', s=50,label='cluster '+ str(i))       
    #각 클러스터의 중심점을 삼각형으로 표시
    for i in range(n_cluster):
        plt.scatter(kmeans.cluster_centers_[i,0], kmeans.cluster_centers_[i,1], 
                    marker='^', color=c_colors[i], edgecolor='w', s=200)
     
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
silhouetteViz(2, X_features_scaled) #클러스터 2개인 경우의 실루엣 score 및 각 클러스터 비중 시각화

In [ ]:
silhouetteViz(3, X_features_scaled) #클러스터 3개인 경우의 실루엣 score 및 각 클러스터 비중 시각화

In [ ]:
silhouetteViz(4, X_features_scaled) #클러스터 4개인 경우의 실루엣 score 및 각 클러스터 비중 시각화

In [ ]:
silhouetteViz(5, X_features_scaled) #클러스터 5개인 경우의 실루엣 score 및 각 클러스터 비중 시각화

In [ ]:
silhouetteViz(6, X_features_scaled) #클러스터 6개인 경우의 실루엣 score 및 각 클러스터 비중 시각화

In [ ]:
clusterScatter(2, X_features_scaled) #클러스터 3개인 경우의 클러스터 데이터 분포 시각화

In [ ]:
clusterScatter(3, X_features_scaled) #클러스터 3개인 경우의 클러스터 데이터 분포 시각화

In [ ]:
clusterScatter(4, X_features_scaled)  #클러스터 4개인 경우의 클러스터 데이터 분포 시각화

In [ ]:
clusterScatter(5, X_features_scaled)  #클러스터 5개인 경우의 클러스터 데이터 분포 시각화

In [ ]:
clusterScatter(6, X_features_scaled)  #클러스터 6개인 경우의 클러스터 데이터 분포 시각화

### 결정된 k를 적용하여 최적의 K-mans 모델 완성

In [ ]:
best_cluster = 4

kmeans = KMeans(n_clusters=best_cluster, random_state=0)
Y_labels = kmeans.fit_predict(X_features_scaled)

In [ ]:
customer_df['ClusterLabel'] = Y_labels

customer_df.head()   #작업 확인용 출력

#### - ClusterLabel이 추가된 데이터를 파일로 저장

In [ ]:
customer_df.to_csv('data/Online_Retail_Customer_Cluster.csv')

## << 클러스터 분석하기 >>

### 1) 각 클러스터의 고객수 

In [ ]:
customer_df.groupby('ClusterLabel')['Customer ID'].count()

### 2) 각 클러스터의 특징

In [ ]:
customer_cluster_df = customer_df.drop(['Freq_log', 'SaleAmount_log', 'ElapsedDays_log'],axis=1, 
                                       inplace=False)

In [ ]:
# 주문 1회당 평균 구매금액 : SaleAmountAvg
customer_cluster_df['SaleAmountAvg'] = customer_cluster_df['SaleAmount']/customer_cluster_df['Freq']
customer_cluster_df.head()

In [ ]:
# 클러스터별 분석
customer_cluster_df.drop(['Customer ID'],axis=1, inplace=False).groupby('ClusterLabel').mean()